In [19]:
import tensorflow as tf
import numpy as np
import os
import re
import io


In [30]:
import pickle
data_dir = "/home/maikfangogoair/tmp/save/"
with open (data_dir + "spectrogram_v1.pkl", 'rb') as fp:
    all_data = pickle.load(fp)

In [31]:
tf.reset_default_graph();
input=tf.placeholder(tf.float32, (None, 99, 161), 'input')

In [32]:
def spectrogram_ConvNet(input):
    """
    architecture as below:
     [Conv2D]<-(weights)
          v
      [BiasAdd]<-(bias)
          v
        [Relu]
          v
      [MatMul]<-(weights)
          v
      [BiasAdd]<-(bias)
          v
      [MatMul]<-(weights)
          v
      [BiasAdd]<-(bias)
          v
      [MatMul]<-(weights)
          v
      [BiasAdd]<-(bias)
      
    output is the logits in shape (batch_size, 12)
    """
    
    reshape_input_1 = tf.reshape(input, [-1, 99, 161, 1], 'reshape_input_1')
    conv_2 = tf.layers.conv2d(reshape_input_1, 64, (3,3), (1,1), 'same', name='conv_2')
    relu_3 = tf.nn.relu(conv_2,'relu_3')
    dropout_prob = tf.placeholder(tf.float32, name='dropout_prob')
    print(dropout_prob)
    relu_3 = tf.nn.dropout(relu_3, dropout_prob, name = 'dropout')
    print(relu_3)
    size=int(relu_3.get_shape()[1]) * int(relu_3.get_shape()[2]) * int(relu_3.get_shape()[3])
    relu_3 = tf.reshape(relu_3, (-1, size), 'reshape_relu_3')
    print(relu_3)
    fc_4 = tf.layers.dense(relu_3, 128,name = 'fc_4')
    fc_5 = tf.layers.dense(fc_4, 128,name = 'fc_5')
    logits = tf.layers.dense(fc_5, 12,name = 'logits')
    
    return logits,dropout_prob

In [33]:
model_name = "spectrogram_ConvNet"
if model_name == "spectrogram_ConvNet":
    logits,dropout_prob = spectrogram_ConvNet(input)

Tensor("dropout_prob:0", dtype=float32)
Tensor("dropout/mul:0", shape=(?, 99, 161, 64), dtype=float32)
Tensor("reshape_relu_3:0", shape=(?, 1020096), dtype=float32)


In [34]:

#training steps
ground_truth_input = tf.placeholder(tf.int64, [None], name='groundtruth_input')
learning_rate_input = tf.placeholder(tf.float32, [], name='learning_rate_input')
cross_entropy_mean = tf.losses.sparse_softmax_cross_entropy(ground_truth_input, logits)
train_step = tf.train.GradientDescentOptimizer(learning_rate_input).minimize(cross_entropy_mean)

predicted_indices = tf.argmax(logits, 1)
correct_prediction = tf.equal(predicted_indices, ground_truth_input)
confusion_matrix = tf.confusion_matrix(ground_truth_input, predicted_indices, num_classes=12)
evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
size=len([x for x in all_data if x["group"] == "training" ])
saver = tf.train.Saver(tf.global_variables())
init = tf.global_variables_initializer()

batch_size = 100
learning_rate = 1e-3
learning_decay = 0.9
learning_decay_period = 1000
eval_every_steps = 1000
training_steps = 10000
print_every = 1

with tf.Session() as sess:
    sess.run(init)
    for i in range(1, training_steps+1):
        start_pos = (i-1) * batch_size % size
        end_pos = (i-1) * batch_size % size + batch_size 
        train_fingerprints = np.asarray([x["spectrogram"] for x in all_data if x["group"] == "training"][start_pos:end_pos])
        train_ground_truth = np.asarray([x["label_idx"] for x in all_data if x["group"] == "training"][start_pos:end_pos]) 
        loss, _ ,accurancy,  = sess.run([cross_entropy_mean, train_step, evaluation_step], \
                feed_dict={
                    input: train_fingerprints,
                    ground_truth_input: train_ground_truth,
                    learning_rate_input: learning_rate * (learning_decay**(i // learning_decay_period)),
                    dropout_prob: 0.5
                })
        if i % print_every == 0:
            print("Step %f, loss: %f, accurancy: %f." % (i, loss, accurancy))
        if  i % eval_every_steps == 0:
            saver.save(sess, data_dir + model_name +".ckpt", global_step=i)
            accurancy = sess.run(evaluation_step, \
                feed_dict={
                    input: np.asarray([x["spectrogram"] for x in all_data if x["group"] == "validation"]),
                    ground_truth_input: np.asarray([x["label_idx"] for x in all_data if x["group"] == "validation"]),
                    dropout_prob: 1.0
                })
            print("Validation accurancy is %f" % accurancy)

Step 1.000000, loss: 3.920009, accurancy: 0.120000.
Step 2.000000, loss: 111.481384, accurancy: 0.080000.
Step 3.000000, loss: 37.752464, accurancy: 0.080000.
Step 4.000000, loss: 26.697796, accurancy: 0.050000.
Step 5.000000, loss: 19.076529, accurancy: 0.110000.
Step 6.000000, loss: 13.675368, accurancy: 0.130000.
Step 7.000000, loss: 13.753356, accurancy: 0.110000.
Step 8.000000, loss: 12.363038, accurancy: 0.100000.
Step 9.000000, loss: 7.840403, accurancy: 0.120000.
Step 10.000000, loss: 5.706493, accurancy: 0.120000.
Step 11.000000, loss: 4.510104, accurancy: 0.080000.
Step 12.000000, loss: 5.326552, accurancy: 0.140000.
Step 13.000000, loss: 3.475159, accurancy: 0.230000.
Step 14.000000, loss: 3.746217, accurancy: 0.050000.
Step 15.000000, loss: 4.638170, accurancy: 0.170000.
Step 16.000000, loss: 4.132074, accurancy: 0.110000.
Step 17.000000, loss: 3.419880, accurancy: 0.170000.
Step 18.000000, loss: 2.630610, accurancy: 0.260000.
Step 19.000000, loss: 3.020654, accurancy: 0.12